In [191]:
import os 
import sys 
import numpy as np 
import random 
import argparse
import cv2
import matplotlib.pyplot as plt
import pickle
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.cluster import KMeans


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

np.random.seed(42)

In [5]:
data_dir = './data/'
class_list = ['Batman', 'Spiderman', 'None']


Width = 256
Height = 512

In [6]:
def get_file_names(path):
    return [os.path.join(path,f) for f in os.listdir(path)]

In [61]:
image_paths = {}
for training_name in class_list:
    dir_= os.path.join(data_dir, training_name)
    class_path = get_file_names(dir_)
    image_paths[training_name] = class_path

In [62]:
label_dict = {'Batman':0, 'Spiderman':1, 'None':2}
image_data = []
for k, v in image_paths.items():
    for im in v: 
        image_data.append((im, label_dict[k]))

In [63]:
np.random.shuffle(image_data)
image_paths, labels = zip(*image_data)

In [76]:
des_list=[]
hog = cv2.HOGDescriptor()

# https://stackoverflow.com/questions/44972099/opencv-hog-features-explanation
# https://answers.opencv.org/question/86023/what-is-the-hog-descriptors-shape/
#

for path in image_paths:
    img = cv2.imread(path)
    resized_image = []
    for i in range(3): 
        resized_image.append(cv2.resize(img[:,:,i], (128,64)))
    resized_image = np.array(resized_image).reshape(128,64,3)
    #creating hog features 
    descriptor = hog.compute(resized_image)
    des_list.append((path,np.array(descriptor)))

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [77]:
path.split('/')[2]

'None'

In [82]:
reshaped_features = []
image_labels = []
BoW = []
BoW_labels = [] 
for i in range(len(des_list)): 
    temp_des = des_list[i][1]
    reshaped_features.append(temp_des.reshape(15*7,-1))
    image_labels.append(label_dict[des_list[i][0].split('/')[2]])
    for j in range(reshaped_features[i].shape[0]):
        BoW.append(reshaped_features[i][j])
        BoW_labels.append(des_list[i][0].split('/')[2])

In [83]:
BoW_array = np.array(BoW)

In [84]:
BoW_array.shape

(22575, 36)

In [177]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

kmeans = KMeans(n_clusters = 256)
kmeans.fit(BoW_array)

preprocessed_image = []

for descriptor in reshaped_features:
    if (descriptor is not None):
        temp_features = []
        for piece in descriptor: 
            histogram = build_histogram(piece.reshape(1,-1), kmeans)  # getting histogram for each piece 
            temp_features.append(histogram)
        temp_features = np.sum(np.array(temp_features), 0)
        preprocessed_image.append(temp_features.reshape(-1,))

In [178]:
preprocessed_image[0]

array([1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 2., 0., 2., 0., 0., 0., 1., 0., 2., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 2., 0., 0., 0., 0., 1., 2., 0., 0., 0., 0., 0., 0., 3., 0.,
       2., 0., 0., 0., 0., 0., 4., 0., 0., 0., 2., 0., 0., 0., 1., 0., 3.,
       0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 3., 0., 0.,
       1., 0., 2., 0., 2., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.,
       0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 2., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 0.

In [ ]:
preprocessed_image[0]

In [168]:
x_train, x_test, y_train, y_test = train_test_split(preprocessed_image, image_labels,
                                                    test_size = .25, random_state = 18)


In [169]:
grid = { 
    'C': [2**(-5), 2**(-4), 2**(-3), 2**(-2), 2**(-1), 1, 2, 4 ,8 ,16],
    'kernel': ['linear', 'poly', 'rbf'],
}

# grid = { 
#     'C': [2**(-5),  2**(-3), 1, 4 ,16],
#     'kernel': ['linear', 'rbf'],
# }

In [170]:
svm_cv = GridSearchCV(estimator=SVC(), param_grid=grid, cv= 5)
svm_cv.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8,
                               16],
                         'kernel': ['linear', 'poly', 'rbf']})

In [171]:
print(svm_cv.best_params_)
best_param = svm_cv.best_params_

{'C': 0.03125, 'kernel': 'poly'}


In [172]:
pickle_file_name = 'best_param_limmited.pkl'
with open(pickle_file_name, 'wb') as handle:
    pickle.dump(svm_cv.best_params_, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [162]:
pickle_file_name = 'best_param_limmited.pkl'
if os.path.exists(pickle_file_name):
    with open(pickle_file_name, 'rb') as handle:
        best_param = pickle.load(handle)

In [173]:
svm_clf = SVC(**best_param)
svm_clf.fit(x_train, y_train)

SVC(C=0.03125, kernel='poly')

In [174]:
y_pred = svm_clf.predict(x_test)

In [175]:
accuracy_score(y_test, y_pred)

0.46296296296296297

In [ ]:
### Preprocess image
def constrastLimit(image):
    img_hist_equalized = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
#    channels = cv2.split(img_hist_equalized)
#    channels[0] = cv2.equalizeHist(channels[0])
#    img_hist_equalized = cv2.merge(channels)
    y, cr, cb = cv2.split(img_hist_equalized)
    y = cv2.equalizeHist(y)
    img_hist_equalized = cv2.merge((y, cr, cb))
    img_hist_equalized = cv2.cvtColor(img_hist_equalized, cv2.COLOR_YCrCb2BGR)
    return img_hist_equalized

def LaplacianOfGaussian(image):
    LoG_image = cv2.GaussianBlur(image, (3,3), 0)           # paramter 
    gray = cv2.cvtColor( LoG_image, cv2.COLOR_BGR2GRAY)
    LoG_image = cv2.Laplacian( gray, cv2.CV_8U,3,3,2)       # parameter
    LoG_image = cv2.convertScaleAbs(LoG_image)
    return LoG_image
    
def binarization(image):
    thresh = cv2.threshold(image,32,255,cv2.THRESH_BINARY)[1]
    #thresh = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    return thresh

def preprocess_image(image):
    image = constrastLimit(image)
    image = LaplacianOfGaussian(image)
    image = binarization(image)
    return image

# Find Signs
def removeSmallComponents(image, threshold):
    #find all your connected components (white blobs in your image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1

    img2 = np.zeros((output.shape),dtype = np.uint8)
    #for every component in the image, you keep it only if it's above threshold
    for i in range(0, nb_components):
        if sizes[i] >= threshold:
            img2[output == i + 1] = 255
    return img2

def findContour(image):
    #find contours in the thresholded image
    cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE    )
#    print (imutils.is_cv2())
#    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = cnts[0]
    return cnts

def contourIsSign(perimeter, centroid, threshold):
    #  perimeter, centroid, threshold
    # # Compute signature of contour
    result=[]
    for p in perimeter:
        p = p[0]
        distance = sqrt((p[0] - centroid[0])**2 + (p[1] - centroid[1])**2)
        result.append(distance)
    max_value = max(result)
    signature = [float(dist) / max_value for dist in result ]
    # Check signature of contour.
    temp = sum((1 - s) for s in signature)
    temp = temp / len(signature)
    if temp < threshold: # is  the sign
        return True, max_value + 2
    else:                 # is not the sign
        return False, max_value + 2

In [ ]:
def cropSign(image, coordinate):
    width = image.shape[1]
    height = image.shape[0]
    top = max([int(coordinate[0][1]), 0])
    bottom = min([int(coordinate[1][1]), height-1])
    left = max([int(coordinate[0][0]), 0])
    right = min([int(coordinate[1][0]), width-1])
    #print(top,left,bottom,right)
    return image[top:bottom,left:right]

In [ ]:
def findLargestSign(image, contours, threshold, distance_theshold):
    max_distance = 0
    coordinate = None
    sign = None
#    print (len(contours))
#    M = [None]*len(contours)
    for c in contours:
#        print (type(c))
#        print (c.shape)
#        print (c.dtype)
        M = cv2.moments(c)
#        M = cv2.moments(np.uint8(c))
        if M["m00"] == 0:
            continue
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        is_sign, distance = contourIsSign(c, [cX, cY], 1-threshold)
        if is_sign and distance > max_distance and distance > distance_theshold:
            max_distance = distance
            coordinate = np.reshape(c, [-1,2])
            left, top = np.amin(coordinate, axis=0)
            right, bottom = np.amax(coordinate, axis = 0)
            coordinate = [(left-2,top-2),(right+3,bottom+1)]
            sign = cropSign(image,coordinate)
    return sign, coordinate

In [186]:
def localization(image, min_size_components, similitary_contour_with_circle, model, current_sign_type): # , count = 0
    original_image = image.copy()
    binary_image = preprocess_image(image)

    binary_image = removeSmallComponents(binary_image, min_size_components)

    binary_image = cv2.bitwise_and(binary_image,binary_image, mask=remove_other_color(image))

    #binary_image = remove_line(binary_image)

    cv2.imshow('BINARY IMAGE', binary_image)
    contours = findContour(binary_image)
    #signs, coordinates = findSigns(image, contours, similitary_contour_with_circle, 15)
    sign, coordinate = findLargestSign(original_image, contours, similitary_contour_with_circle, 15)
    
    text = ""
    sign_type = -1
    i = 0

#     if sign is not None:
#         sign_type = getLabel(model, sign)
#         sign_type = sign_type if sign_type <= 8 else 8
#         text = SIGNS[sign_type]
#         cv2.imwrite(str(count)+'_'+text+'.png', sign)

    if sign_type > 0 and sign_type != current_sign_type:        
        cv2.rectangle(original_image, coordinate[0],coordinate[1], (0, 255, 0), 1)
        font = cv2.FONT_HERSHEY_PLAIN
        cv2.putText(original_image,text,(coordinate[0][0], coordinate[0][1] -15), font, 1,(0,0,255),2,cv2.LINE_4)
    return coordinate, original_image, sign_type, text

In [198]:
parser = argparse.ArgumentParser(description="NLP Assignment Command Line")

parser.add_argument(
  '--file_name',
  default= "./MVI_1049.avi",
  help= "Video to be analyzed"
  )

parser.add_argument(
  '--min_size_components',
  type = int,
  default= 300,
  help= "Min size component to be reserved"
  )


parser.add_argument(
  '--similitary_contour_with_circle',
  type = float,
  default= 0.65,
  help= "Similitary to a circle"
  )

# args = parser.parse_args()

_StoreAction(option_strings=['--similitary_contour_with_circle'], dest='similitary_contour_with_circle', nargs=None, const=None, default=0.65, type=<class 'float'>, choices=None, help='Similitary to a circle', metavar=None)

In [193]:
img = cv2.imread('data/test_comic_locations/batman1.jpeg')

In [201]:
frame = cv2.resize(img, (640,480))

# print("Frame:{}".format(count))
#image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
coordinate, image, sign_type, text = localization(frame, 300, 0.65, svm_clf, current_sign)
if coordinate is not None:
    cv2.rectangle(image, coordinate[0],coordinate[1], (255, 255, 255), 1)

NameError: name 'current_sign' is not defined